## Testing Batching

This notebook sends a controlled number of HTTP events to a Nuclio function that is configured for batching and then validates how the function actually batched and processed them. It collects all responses, builds a pandas DataFrame, and checks that the first batch reached the configured batch size, was handled together, and got the same end time. It then verifies that the remaining event was flushed in a second batch and that this second batch happened after the batching timeout window. The notebook also prints the batch sizes and per worker distribution so you can see how Nuclio spread the work across workers.

In [1]:
import mlrun
import time

In [2]:
project = mlrun.get_or_create_project("batching-nuclio",user_project=True,allow_cross_project=True)

> 2025-11-09 13:05:11,303 [info] Project loaded successfully: {"project_name":"batching-nuclio-guyl"}


In [3]:
%%writefile batching-nuclio.py
import nuclio_sdk
import json
from datetime import datetime

BATCH_NUM = 0  # start from 0

def handler(context, batch: list[nuclio_sdk.Event]):
    global BATCH_NUM
    context.logger.info_with("Got batched event", size=len(batch))

    BATCH_NUM += 1   # each batch gets the next number
    this_batch_num = BATCH_NUM

    response_msg = process_batch(batch)
    responses = []

    for item in batch:
        body = item.body
        try:
            if isinstance(body, bytes):
                body = body.decode("utf-8")
            parsed = json.loads(body)
        except Exception:
            parsed = body

        resp_body = {
            "body": parsed,
            "batch_num": this_batch_num,
            "batch_ts": datetime.now().isoformat(timespec="milliseconds"),
            "msg": response_msg,
            "worker_id": context.worker_id,
            "end_time": datetime.now().isoformat(),
        }

        responses.append(
            nuclio_sdk.Response(
                body=json.dumps(resp_body),
                headers={"Content-Type": "application/json"},
                content_type="application/json",
                status_code=200,
                event_id=item.id,
            )
        )
    return responses

def process_batch(event_list: list[nuclio_sdk.Event]):
    return "hello"


Overwriting batching-nuclio.py


## Test Batch Size #1
 Testing with the following parameters: `11 events`, `batch size 10`, `1 workers`, `timeout 5s`, test 11 events process 10 in one batch and wait 5 seconds for the 11th event.

In [4]:
nuclio_batch2 = project.set_function(func="batching-nuclio.py",name="batching-nuclio",image="mlrun/mlrun",kind="nuclio")
nuclio_batch2.spec.handler = "batching-nuclio:handler"

In [5]:
nuclio_batch2.with_http(workers=1, worker_timeout=30)
nuclio_batch2.spec.max_replicas = 1
nuclio_batch2.spec.config["spec.triggers.http"]["batch"]={"mode": "enable","batchSize":10, "timeout":"5s","eventTimeout": "60s"}

> 2025-11-09 13:05:11,329 [warning] Adding HTTP trigger despite the default HTTP trigger creation being disabled


In [6]:
addr = nuclio_batch2.deploy()
print("URL:", addr)

> 2025-11-09 13:05:11,338 [info] Starting remote function deploy
2025-11-09 13:05:11  (info) Deploying function
2025-11-09 13:05:11  (info) Building
2025-11-09 13:05:11  (info) Staging files and preparing base images
2025-11-09 13:05:11  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2025-11-09 13:05:11  (info) Building processor image
2025-11-09 13:06:17  (info) Build complete
2025-11-09 13:06:25  (info) Function deploy complete
> 2025-11-09 13:06:32,232 [info] Successfully deployed function: {"external_invocation_urls":["batching-nuclio-guyl-batching-nuclio.default-tenant.app.cust-cs-il.iguazio-cd0.com/"],"internal_invocation_urls":["nuclio-batching-nuclio-guyl-batching-nuclio.default-tenant.svc.cluster.local:8080"]}
URL: http://batching-nuclio-guyl-batching-nuclio.default-tenant.app.cust-cs-il.iguazio-cd0.com/


In [7]:
import requests
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import math

URL = addr
NUM_EVENTS = 11
BATCH_SIZE = 10
EXPECTED_TIMEOUT = 5      # seconds
TIME_TOLERANCE = 1.0      # seconds, to allow for scheduler jitter

def fetch(i, timeout=20):
    payload = {"test": i, "send_at": datetime.now().isoformat()}
    r = requests.post(URL, json=payload, timeout=timeout)
    return i, r.status_code, r.text

# send events
results = []
with ThreadPoolExecutor(max_workers=10) as ex:
    futs = [ex.submit(fetch, i) for i in range(NUM_EVENTS)]
    for f in as_completed(futs):
        results.append(f.result())

# build df
rows = []
for i, status, body in results:
    data = json.loads(body)
    sent_payload = data["body"]
    if isinstance(sent_payload, str):
        try:
            sent_payload = json.loads(sent_payload)
        except Exception:
            sent_payload = {}
    rows.append(
        {
            "i": i,
            "status": status,
            "batch_num": data["batch_num"],
            "send_at": sent_payload.get("send_at"),
            "end_time": data["end_time"],
        }
    )

df = pd.DataFrame(rows).sort_values("i").reset_index(drop=True)

# Verify that the number of responses matches the number of events sent
assert len(df) == NUM_EVENTS, f"Expected {NUM_EVENTS}, got {len(df)}"
assert (df["status"] == 200).all(), f"Non 200 rows: {df[df['status'] != 200]}"

# Convert time fields to datetime objects and normalize end_time to 0.1-second precision
# This allows consistent comparison of timing between events.
df["send_at"] = pd.to_datetime(df["send_at"])
df["end_time"] = pd.to_datetime(df["end_time"])
df["end_time_0_1s"] = df["end_time"].dt.round("100ms")

# 1. Validate the first batch behavior
# Confirm that the first batch reached exactly the configured batch size
# and that the first 10 (batch size) rows all belong to the same batch number.

first_batch_num = df.loc[0, "batch_num"]
first_batch_df = df[df["batch_num"] == first_batch_num].sort_values("i")

assert len(first_batch_df) == BATCH_SIZE, (
    f"First batch size is {len(first_batch_df)}, expected {BATCH_SIZE}"
)

assert (df.loc[:BATCH_SIZE-1, "batch_num"] == first_batch_num).all(), \
    f"First {BATCH_SIZE} rows are not in the same batch"

# Ensure all first-batch events were processed together,
# by confirming they share the same rounded end_time.
first_batch_times = first_batch_df["end_time_0_1s"].unique()
assert len(first_batch_times) == 1, f"First batch has multiple times: {first_batch_times}"

# 2. Validate the second batch trigger
# Check that the remaining event was processed in a separate batch
# and that this batch has a distinct batch number and correct size 1.
second_row = df.loc[BATCH_SIZE]
second_batch_num = second_row["batch_num"]
assert second_batch_num != first_batch_num, \
    f"Second batch row has same batch_num as first: {second_batch_num}"

# second batch should have the remaining events
second_batch_df = df[df["batch_num"] == second_batch_num]
expected_second_batch_size = NUM_EVENTS - BATCH_SIZE
assert len(second_batch_df) == expected_second_batch_size, (
    f"Second batch size is {len(second_batch_df)}, expected {expected_second_batch_size}"
)

# 3. Verify batching timeout behavior
# The second batch should be processed after approximately the batching timeout period.
# Measure the gap between the first and second batch end_time values.
first_batch_time = first_batch_df["end_time"].min()
second_batch_time = second_batch_df["end_time"].min()
diff = (second_batch_time - first_batch_time).total_seconds()

assert abs(diff - EXPECTED_TIMEOUT) <= TIME_TOLERANCE, (
    f"Second batch did not fire near timeout. "
    f"diff={diff:.2f}s, expected about {EXPECTED_TIMEOUT}s"
)

# 4. Ensure the batches were distinct in time
# Confirm that the rounded end_time values for the first and second batch differ,
# proving they were executed as separate batch windows.
assert second_batch_df["end_time_0_1s"].iloc[0] != first_batch_times[0], \
    "Second batch rounded time equals first batch time"

Difference:  {5.03502}
✅ batching timing test passed
     i  status  batch_num                    send_at  \
0    0     200          1 2025-11-09 13:06:32.308469   
1    1     200          1 2025-11-09 13:06:32.308974   
2    2     200          1 2025-11-09 13:06:32.309344   
3    3     200          1 2025-11-09 13:06:32.309766   
4    4     200          1 2025-11-09 13:06:32.319822   
5    5     200          1 2025-11-09 13:06:32.322943   
6    6     200          1 2025-11-09 13:06:32.323659   
7    7     200          1 2025-11-09 13:06:32.335908   
8    8     200          1 2025-11-09 13:06:32.339862   
9    9     200          1 2025-11-09 13:06:32.349957   
10  10     200          2 2025-11-09 13:06:32.410415   

                     end_time           end_time_0_1s  
0  2025-11-09 13:06:32.408184 2025-11-09 13:06:32.400  
1  2025-11-09 13:06:32.408143 2025-11-09 13:06:32.400  
2  2025-11-09 13:06:32.408170 2025-11-09 13:06:32.400  
3  2025-11-09 13:06:32.408195 2025-11-09 13:06:32.